In [ ]:
#####CTRL+F and search for 'CTRL+START', to go to the starting cell block.
#ALL THE EXPLANATION ARE COMMENTS IN CAPS LOCK

#THIS CODE BLOCK MAKES SURE THAT THE NOTEBOOK HAS THE NECESSARY LIBRARIES. BY DEFAULT, EVERY SESSION IN COLAB IS A FRESH PYTHON
# ENVIRONMENT, SO TO MAKE SURE THAT THE NECESSARY LIBRARIES ARE PRESENT THIS CODE BLOCK IS RUN
#IVE MADE SURE THAT THE LIBRARIES WILL ONLY BE INSTALLED ONCE EVERY SESSION, RESTARTING THE NOTEBOOOK DOES NOT AFFECT IT.
#MAKE SURE TO NEVER DISCONNECT AND DELETE RUNTIME, AS IT WILL REMOVE EVERY CUSTOM INSTALLATION OF THE LIBRARIES, AND CONNECT THE NOTEBOOK TO A FRESH
#PYTHON INSTALLATION.

from IPython.display import clear_output as clear,Image
import json,os,time,tarfile,shutil,numpy as np, nibabel
from glob import glob

from nibabel import load,Nifti1Image,save
from tqdm import tqdm
from matplotlib import pyplot as plt
##############JUST PLOTTING

try:
  from humanfriendly import format_timespan, format_size
except ModuleNotFoundError:
  !pip install humanfriendly
  from humanfriendly import format_timespan, format_size


try:
  import pydicom
except ModuleNotFoundError:
  !pip install pydicom
  import pydicom


try:
  import dicom2nifti
except ModuleNotFoundError:
  !pip install dicom2nifti
  import dicom2nifti


try:
  import SimpleITK as sitk
except ModuleNotFoundError:
  !pip install simpleitk
  import SimpleITK as sitk


try:
  import wget
except ModuleNotFoundError:
  !pip install wget
  import wget

try:
  import monai
  from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
  )
  from monai.data import DataLoader, Dataset, CacheDataset
  from monai.utils import set_determinism
  from monai.utils import first

except ModuleNotFoundError:
  !pip install monai
  !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
  from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
  from monai.data import DataLoader, Dataset, CacheDataset
  from monai.utils import set_determinism
  from monai.utils import first


import numpy as np, nibabel
from matplotlib import pyplot as plt
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism
from monai.utils import first
##############JUST PLOTTING

#SOME IMPORTS


import numpy as np, nibabel
from matplotlib import pyplot as plt
from monai.transforms import(
  Compose,
  AddChanneld,
  LoadImaged,
  Resized,
  ToTensord,
  Spacingd,
  Orientationd,
  ScaleIntensityRanged,
  CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism
from monai.utils import first
from monai.utils import first, set_determinism
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    Activations,
)

from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.data import CacheDataset, DataLoader, Dataset
from monai.inferers import sliding_window_inference
from monai.utils import first
import matplotlib.pyplot as plt
import torch
import os
import numpy as np
from monai.losses import DiceLoss
from tqdm import tqdm


main_path = '/content/drive/MyDrive/thesis_dataset/'
livpath = main_path+'Liver/Task03_Liver/'
mypath = main_path+'test_path/'
trpath = main_path+'imagesTr_path'
labels = main_path+'labels_path'
tspath = main_path+'imagesTs_path'
folder = 'imagesTr'
sixtyfournifties = main_path+'groups_of_64_nifti/'+folder+'/'

separator = ' —— '
sep = ' — '
empty_list = ['',""," ",' ']

main_path = '/content/drive/MyDrive/thesis_dataset/'
livpath = main_path+'Liver/Task03_Liver/'
mypath = main_path+'test_path/'

trpath = main_path+'imagesTr_path'
labels = main_path+'labels_path'
tspath = main_path+'imagesTs_path'
traindir = main_path+'TRAINER/'
tsseg = traindir+'TestSegmentation/'
tsvol = traindir+'TestVolumes/'
trseg = traindir+'TrainSegmentation/'
trvol = traindir+'TrainVolumes/'
data_dir = traindir
model_dir = main_path+'MODELscratch/'
kfold = main_path+'KFold/'



clear()
my_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('CHOOSING',my_device,'to do the training')

clear()

In [ ]:

#EASE OF USE, THIS TO_JSON FUNCTION DUMPS ALL THE DICTONARY INTO A JSON FILE. THIS IS MAINLY FOR STORING DATA.
## THE FUNCTION NAMES ARE SELF-EXPLANATORY

def to_json(dic, name, string):
  # print('to_json:',string,sep,name)c
  path = main_path
  a = 1
  if string == 'write':

    if dic == {}:
      print('Given Dictionary is empty. Check again!!')
      return False
    if a == 1:

      with open(path+name+'.json','w') as fp:
        json.dump(dic,fp)
    else:
      print('Did not save anything.')
      return False

    # print('saved '+name+'.json')
    print('saved',sep,name,'('+getfilesize(path+name+'.json',1)+')')

  if string == 'read':
    with open(path+name+'.json','r') as fp:
      data = json.load(fp)

    return data


def getfilesize(the_file,the_type):
  file_size = os.path.getsize(the_file)
  actual_file_size = format_size(file_size)

  la_output = the_file.split('/')[-1]+sep+actual_file_size

  if the_type not in empty_list:
    if the_type == 1:
      return la_output.split(sep)[-1]
    else:
      return file_size
  else:
    print(la_output)

def get_dir_size(path='.'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)

    return total



def timenow():
  from datetime import date as datetimedate, datetime, timedelta
  import pytz
  return datetime.now(pytz.timezone("Etc/GMT-6")).strftime('%d %B %Y, %I:%M:%S %p')
# timenow()
def epoch_checker():
  from matplotlib import pyplot as plt
  train_loss = np.load(os.path.join(model_dir, 'loss_train.npy'))
  train_metric = np.load(os.path.join(model_dir, 'metric_train.npy'))
  test_loss = np.load(os.path.join(model_dir, 'loss_test.npy'))
  test_metric = np.load(os.path.join(model_dir, 'metric_test.npy'))
  fig,ax = plt.subplots(figsize=(6,12),nrows=2,ncols=1,)

  a, = ax[0].plot(test_loss,'o-',label='test_loss')
  b, = ax[0].plot(train_loss,'x--',label='train_loss')
  # ax.grid()
  ax[0].set_title('DICE loss')
  ax[0].legend(handles=[a,b])
  # plt.legend()
  ax[0].grid()
  # plt.subplot(2,1,2)

  a, = ax[1].plot(test_metric,'o-',label='test_metric')
  b, = ax[1].plot(train_metric,'x--',label='train_metric')

  ax[1].legend(handles=[a,b])
  ax[1].set_title('DICE metric')
  plt.suptitle('\n\nepoch'+sep+str(len(train_loss))+'\n'+timenow())
  ax[1].grid()
  # if os.path.exists(model_dir+'graphs/') is False:
  #   os.mkdir(model_dir+'graphs/')
  # plt.legend()

  plt.savefig(model_dir+'graphs/'+timenow()+'.png',bbox_inches='tight')
  # plt = 0

# epoch_checker()

#THIS CODE BLOCK IS HOW WE TRAINED THE MODEL

In [ ]:
####THIS TAKES THE TENSORS DATA AND THEN TRAINS THE MODEL, AND THE DICE_METRIC AND CALCULATED WEIGHTS ARE THERE TO TUNE THE ERROR THRESHOLD OF THE MODELS OUTPUT

def dice_metric(predicted, target):
    '''
    In this function we take `predicted` and `target` (label) to calculate the dice coeficient then we use it
    to calculate a metric value for the training and the validation.
    '''
    dice_value = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
    value = 1 - dice_value(predicted, target).item()
    return value

def calculate_weights(val1, val2):
    '''
    In this function we take the number of the background and the forgroud pixels to return the `weights`
    for the cross entropy loss values.
    '''
    count = np.array([val1, val2])
    summ = count.sum()
    weights = count/summ
    weights = 1/weights
    summ = weights.sum()
    weights = weights/summ
    return torch.tensor(weights, dtype=torch.float32)

def train(model, data_in, loss, optim, max_epochs, model_dir, test_interval=1 , device=torch.device("cuda:0")):
    best_metric = -1
    best_metric_epoch = -1
    try:
      save_loss_train = list(np.load(os.path.join(model_dir, 'loss_train.npy')))
      save_loss_test = list(np.load(os.path.join(model_dir, 'loss_test.npy')))
      save_metric_train = list(np.load(os.path.join(model_dir, 'metric_train.npy')))
      save_metric_test = list(np.load(os.path.join(model_dir, 'metric_test.npy')))
    except FileNotFoundError:
      save_loss_train = []
      save_loss_test = []
      save_metric_train = []
      save_metric_test = []
    train_loader, test_loader = data_in


    for epoch in range(max_epochs):
        # clear()
        # try:
        #   plt= 0
        #   epoch_checker()
        # except FileNotFoundError:
        #   print('Not doing the EPOCH CHECKER')
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")

        model.train()
        try:
          train_epoch_loss = save_loss_train[-1]
          epoch_metric_train = save_metric_train[-1]
        except IndexError:
          train_epoch_loss = 0
          epoch_metric_train = 0

        train_step = 0

        for batch_data in train_loader:
            clear()
            train_step += 1

            volume = batch_data["vol"]
            label = batch_data["seg"]
            label = label != 0
            volume, label = (volume.to(device), label.to(device))

            optim.zero_grad()
            outputs = model(volume)

            train_loss = loss(outputs, label)

            train_loss.backward()
            optim.step()

            train_epoch_loss += train_loss.item()
            print(
                f"{train_step}/{len(train_loader) // train_loader.batch_size}, "
                f"Train_loss: {train_loss.item():.4f}")

            train_metric = dice_metric(outputs, label)
            epoch_metric_train += train_metric
            print(f'Train_dice: {train_metric:.4f}')
            !nvidia-smi
            # time.sleep(1)
            # clear()

        clear()
        print('-'*20)

        train_epoch_loss /= train_step
        print(f'Epoch_loss: {train_epoch_loss:.4f}')
        save_loss_train.append(train_epoch_loss)
        np.save(os.path.join(model_dir, 'loss_train.npy'), save_loss_train)

        epoch_metric_train /= train_step
        print(f'Epoch_metric: {epoch_metric_train:.4f}')

        save_metric_train.append(epoch_metric_train)
        np.save(os.path.join(model_dir, 'metric_train.npy'), save_metric_train)


        if (epoch + 1) % test_interval == 0:

            model.eval()
            with torch.no_grad():
                try:
                  test_epoch_loss = save_loss_test[-1]
                  epoch_metric_test = save_metric_test[-1]
                except IndexError:
                  test_epoch_loss = 0
                  epoch_metric_test = 0

                test_metric = 0

                test_step = 0

                for test_data in test_loader:
                    # clear()
                    test_step += 1

                    test_volume = test_data["vol"]
                    test_label = test_data["seg"]
                    test_label = test_label != 0
                    test_volume, test_label = (test_volume.to(device), test_label.to(device),)

                    test_outputs = model(test_volume)

                    test_loss = loss(outputs, test_label)
                    test_epoch_loss += test_loss.item()
                    test_metric = dice_metric(test_outputs, test_label)
                    epoch_metric_test += test_metric




                test_epoch_loss /= test_step
                print(f'test_loss_epoch: {test_epoch_loss:.4f}')

                # clear()
                save_loss_test.append(test_epoch_loss)
                np.save(os.path.join(model_dir, 'loss_test.npy'), save_loss_test)

                epoch_metric_test /= test_step
                print(f'test_dice_epoch: {epoch_metric_test:.4f}')
                save_metric_test.append(epoch_metric_test)
                np.save(os.path.join(model_dir, 'metric_test.npy'), save_metric_test)
                clear()
                print('checking epoch...')
                time.sleep(1)
                epoch_checker()
                clear()


                if epoch_metric_test > best_metric:
                    best_metric = epoch_metric_test
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), os.path.join(
                        model_dir, "best_metric_model.pth"))

                print(
                    f"current epoch: {epoch + 1} current mean dice: {test_metric:.4f}"
                    f"\nbest mean dice: {best_metric:.4f} "
                    f"at epoch: {best_metric_epoch}"
                  )



    print(
        f"train completed, best_metric: {best_metric:.4f} "
        f"at epoch: {best_metric_epoch}")



def calculate_pixels(data):
    val = np.zeros((1, 2))

    for batch in tqdm(data):
        batch_label = batch["seg"] != 0
        _, count = np.unique(batch_label, return_counts=True)

        if len(count) == 1:
            count = np.append(count, 0)
        val += count

    print('The last values:', val)
    return val


In [ ]:
#PREDICTING LIVER MASK FOR TEST FILES WITHOUT SEGMENTATION ------- SELF EXPLANATORY
#PREDICTING LIVER MASK FOR NEW FILES ---- TAKES IN IMAGES WITHIN THE TRAINING FOLDER THAT WERE NOT USED IN TRAINING.
#SO AS FAR AS THE MODEL IS CONCERNED THOSE ARE NEW FILES, WE TEST THE MODELS OUTPUT AGAINST THE ACTUAL SEGMENTATION
#AND VISUALLY WE TRY TO DETERMINE IF THE MODEL IS BEHAVING CORRECTLY.

#SHOWS PROCESSED IMAGE PIPELINE FOR A RANDOM IMAGE -- SELF EXPLANATORY

#CORRECTS THE ORIENTATION OF THE IMAGE
def correct_orient(a):
  a = np.fliplr(a)
  a = np.rot90(a,3)
  return a

def predicting_liver_mask_for_test_files_without_segmentation(test_files,start_slice,end_slice,colormap='grey',model_path=main_path+'/MODELscratch/best_metric_model.pth'):
# colormap = 'gray'
  file_name = test_files[0]['vol'].split(tsvol)[-1]
  print('Validating against',test_files[0]['vol'].split(trvol)[-1])
  print('from slice',start_slice,'to',end_slice)


  test_transforms = Compose(
      [
          LoadImaged(keys=["vol", "seg"],allow_missing_keys=True),
          AddChanneld(keys=["vol", "seg"],allow_missing_keys=True),
          Spacingd(keys=["vol", "seg"],allow_missing_keys=True, pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
          Orientationd(keys=["vol", "seg"],allow_missing_keys=True, axcodes="RAS"),
          ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True),
          CropForegroundd(keys=['vol', 'seg'], source_key='vol',allow_missing_keys=True),
          Resized(keys=["vol", "seg"],allow_missing_keys=True, spatial_size=[128,128,64]),
          ToTensord(keys=["vol", "seg"],allow_missing_keys=True),
      ]
  )

  test_ds = Dataset(data=test_files, transform=test_transforms)

  # test_ds = Dataset(data=test_files)
  test_loader = DataLoader(test_ds, batch_size=1)
  device = torch.device(my_device)
  model = UNet(
      dimensions=3,
      in_channels=1,
      out_channels=2,
      channels=(16, 32, 64, 128, 256),
      strides=(2, 2, 2, 2),
      num_res_units=2,
      norm=Norm.BATCH,
  ).to(device)
  if model_path not in empty_list:
    model.load_state_dict(torch.load(
        model_path,map_location=torch.device(my_device)))
    model.eval()

  if model_path in empty_list:
    model_path = '**no models were used**'
  sw_batch_size = 4
  roi_size = (128, 128, 64)
  with torch.no_grad():
      test_patient = first(test_loader)
      t_volume = test_patient['vol']
      # t_segmentation = test_patient['seg']

      test_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
      # tarp = test_outputs
      sigmoid_activation = Activations(sigmoid=True)
      test_outputs = sigmoid_activation(test_outputs)
      # test_outputs = test_outputs > 0.997
      test_outputs = test_outputs > .75
      # import cv2
      for i in range(start_slice,end_slice+1):
          # plot the slice [:, :, 80]
          processed_slice = test_patient["vol"][0, 0, :, :, i].numpy()
          # target = test_patient["seg"][0, 0, :, :, i].numpy() > 0
          target= np.ones([128,128],dtype='float32')
          actual_slice = nibabel.load(tsvol+file_name).get_fdata()[:,:,i]
          prediction = np.array(test_outputs.detach().cpu()[0,1,:,:,i],dtype='float32')
          difference = np.array(np.logical_xor(processed_slice,prediction),dtype='float32')
          # difference = 1-(2*(np.logical_and(target,prediction)/np.logical_or(target,prediction)))

          processed_slice = correct_orient(processed_slice)
          target = correct_orient(target)
          actual_slice = correct_orient(actual_slice)
          prediction = correct_orient(prediction)
          difference = correct_orient(difference)

          try:
            accuracy = round((1-(abs(np.count_nonzero(difference))/np.count_nonzero(target)))*100,3)
            print('xor accuracy:',accuracy)
            # accuracy = round(dice_coef(prediction,target)*100,3)
            accuracy = round(np.sum(prediction[target==1]*2.0/(np.sum(prediction)+np.sum(target)))*100,3)
            print('dice coeff:',accuracy)

          except ZeroDivisionError:
            if np.count_nonzero(target)==0 and np.count_nonzero(prediction) != 0:
              accuracy = 0
            if np.count_nonzero(target)!=0  and np.count_nonzero(prediction) == 0:
              accuracy = 0
            if np.count_nonzero(target)==0 and np.count_nonzero(prediction) == 0:
              accuracy = 100


          # plt.rc('lines',linewidth=1.6)

          fig, ax = plt.subplots(figsize=(30,12))
          plt.rc('font',family='serif',size=35)
          COLOR = [0,0,0]
          # ax.tick_params(size=14)
          # ax.yticks(fontsize=14)
          plt.axis('on')
          plt.rcParams['axes.labelcolor'] = COLOR
          plt.rcParams['xtick.labelsize'] = 26
          plt.rcParams['ytick.labelsize'] = 0
          plt.rcParams['xtick.color'] = COLOR
          plt.rcParams['ytick.color'] = COLOR

          # plt.rcParams['text.color'] = [1,1,1]
          plt.subplot(1,5,1)
          plt.title(f"SLICE\n")
          plt.imshow(actual_slice,cmap=colormap)
          plt.xticks([0,actual_slice.shape[0]])
          plt.yticks([0,0])

          plt.subplot(1, 5, 2)
          plt.title(f"PROCESSED\nSLICE")
          plt.imshow(processed_slice,cmap=colormap)
          plt.xticks([0,processed_slice.shape[0]])
          plt.yticks([0, 0])

          plt.subplot(1, 5, 4)
          plt.title(f"OVERLAY\n")
          mask = prediction
          plt.imshow(processed_slice,cmap=colormap)
          plt.imshow(np.ma.masked_where(mask==0,mask),cmap='jet')


          plt.xticks([0,processed_slice.shape[0]])
          plt.yticks([0,0])

          plt.subplot(1, 5, 3)
          plt.title(f"PREDICTION\n")
          plt.imshow(prediction,cmap=colormap)
          plt.xticks([0,prediction.shape[0]])
          plt.yticks([0,0])

          # clear()
          plt.subplot(1,5,5)
          plt.title(f"SEGMENTED\nPART")
          plt.imshow(processed_slice, cmap='gray', alpha=1.0*(prediction>0))
          plt.xticks([0,processed_slice.shape[0]])
          plt.yticks([0,0])
          # plt.suptitle('TEST FILE: '+tsvol+file_name+'\nSLICE NUMBER: '+str(i)+'\n'+
          #             'MODEL USED: '+model_path.split(main_path)[-1]+'\n'+
          #             'DEVICE: '+my_device.upper()+'\n'+
          #             # 'THRESHOLD: '+str(threshold_of_error)+' [0 --> 1]\n'+
          #             # 'ACCURACY: '+str(accuracy)+'%\n'+
          #             'TIMESTAMP: '+timenow(),ha='left',va='top',x=0.11,y=.89)
          if accuracy > 10 and accuracy < 100:
            # clear()
            file_name = model_path.split(main_path)[-1].split('/')[2]+' [segmenting]'
            print(file_name)
            plt.savefig(main_path+'TESTED_DATA/'+file_name+sep+timenow()+'.png',bbox_inches='tight',dpi=600)
            plt.show()
            print('SAVED VALIDATION....')

          # plt.savefig(main_path+'TESTED_DATA/TESTVOLUME '+timenow()+'.png',bbox_inches='tight',dpi=600)
          # plt.show()
          # print('SAVED VALIDATION....')


#####THIS


def predicting_liver_mask_for_new_files(file_index,start_slice,end_slice,colormap='gray',threshold_of_error=0.53,model_path=main_path+'/MODELscratch/best_metric_model.pth'):


  my_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  print('CHOOSING',my_device,'to do the validation')

  in_dir = traindir

  path_train_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))[0:index]
  path_train_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))[0:index]

  path_test_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))[index:]
  path_test_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))[index:]

  train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_train_volumes, path_train_segmentation)]
  test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]
  # test_files = test_files
  # print(test_files)
  # test_files = [{'vol':tsvol+'liver_132_0.nii.gz','seg':trseg+'liver_0_0.nii.gz'}]

  test_files = test_files[file_index-1:file_index]
  file_name = test_files[0]['vol'].split(trvol)[-1]
  print('Validating against',test_files[0]['vol'].split(trvol)[-1])
  print('from slice',start_slice,'to',end_slice)


  test_transforms = Compose(
      [
          LoadImaged(keys=["vol", "seg"]),
          AddChanneld(keys=["vol", "seg"]),
          Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
          Orientationd(keys=["vol", "seg"], axcodes="RAS"),
          ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True),
          CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
          Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),
          ToTensord(keys=["vol", "seg"]),
      ]
  )

  test_ds = Dataset(data=test_files, transform=test_transforms)

  # test_ds = Dataset(data=test_files)
  test_loader = DataLoader(test_ds, batch_size=1)
  device = torch.device(my_device)
  model = UNet(
      dimensions=3,
      in_channels=1,
      out_channels=2,
      channels=(16, 32, 64, 128, 256),
      strides=(2, 2, 2, 2),
      num_res_units=2,
      norm=Norm.BATCH,
  ).to(device)
  if model_path not in empty_list:
    model.load_state_dict(torch.load(
        model_path,map_location=torch.device(my_device)))
    model.eval()

  if model_path in empty_list:
    model_path = '**no models were used**'
  sw_batch_size = 4
  roi_size = (128, 128, 64)
  with torch.no_grad():
      test_patient = first(test_loader)
      t_volume = test_patient['vol']
      t_segmentation = test_patient['seg']

      test_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
      # tarp = test_outputs
      sigmoid_activation = Activations(sigmoid=True)
      test_outputs = sigmoid_activation(test_outputs)
      # test_outputs = test_outputs > 0.997
      test_outputs = test_outputs > .75
      # import cv2
      for i in range(start_slice,end_slice+1):
          # plot the slice [:, :, 80]
          processed_slice = test_patient["vol"][0, 0, :, :, i].numpy()
          target = test_patient["seg"][0, 0, :, :, i].numpy() > 0
          actual_slice = nibabel.load(trvol+file_name).get_fdata()[:,:,i]
          prediction = np.array(test_outputs.detach().cpu()[0,1,:,:,i],dtype='float32')
          difference = np.array(np.logical_xor(prediction,target),dtype='float32')
          # difference = 1-(2*(np.logical_and(target,prediction)/np.logical_or(target,prediction)))


          processed_slice = correct_orient(processed_slice)
          target = correct_orient(target)
          actual_slice = correct_orient(actual_slice)
          prediction = correct_orient(prediction)
          difference = correct_orient(difference)

          try:
            accuracy = round((1-(abs(np.count_nonzero(difference))/np.count_nonzero(target)))*100,3)
            print('xor accuracy:',accuracy)
            # accuracy = round(dice_coef(prediction,target)*100,3)
            accuracy = round(np.sum(prediction[target==1]*2.0/(np.sum(prediction)+np.sum(target)))*100,3)
            print('dice coeff:',accuracy)

          except ZeroDivisionError:
            if np.count_nonzero(target)==0 and np.count_nonzero(prediction) != 0:
              accuracy = 0
            if np.count_nonzero(target)!=0  and np.count_nonzero(prediction) == 0:
              accuracy = 0
            if np.count_nonzero(target)==0 and np.count_nonzero(prediction) == 0:
              accuracy = 100


          # plt.rc('lines',linewidth=1.6)

          fig, ax = plt.subplots(figsize=(30,12))
          plt.rc('font',family='serif',size=35)
          COLOR = [0,0,0]
          # ax.tick_params(size=14)
          # ax.yticks(fontsize=14)
          plt.axis('on')
          plt.rcParams['axes.labelcolor'] = COLOR
          plt.rcParams['xtick.labelsize'] = 26
          plt.rcParams['ytick.labelsize'] = 0
          plt.rcParams['xtick.color'] = COLOR
          plt.rcParams['ytick.color'] = COLOR

          plt.subplot(1,5,1)
          plt.title(f"SLICE\n")
          plt.imshow(actual_slice,cmap=colormap)
          plt.xticks([0, actual_slice.shape[0]])
          plt.yticks([0, actual_slice.shape[0]])

          plt.subplot(1, 5, 2)
          plt.title(f"PROCESSED\nSLICE")
          plt.imshow(processed_slice,cmap=colormap)
          plt.xticks([0, processed_slice.shape[0]])
          plt.yticks([0, 0])

          plt.subplot(1, 5, 3)
          plt.title(f"TARGET\n")
          plt.imshow(target,cmap=colormap)
          plt.xticks([0, target.shape[0]])
          plt.yticks([0, 0])
          # plt.imshow(nibabel.load(test_files[0]['seg']).get_fdata()[:,:,i])
          plt.subplot(1, 5, 4)
          plt.title(f"PREDICTION\n")
          plt.imshow(prediction,cmap=colormap)
          plt.xticks([0, prediction.shape[0]])
          plt.yticks([0, 0])

          # clear()
          plt.subplot(1,5,5)
          plt.title(f"DIFFERENCE\n")
          plt.imshow(difference,cmap=colormap)
          plt.xticks([0, difference.shape[0]])
          plt.yticks([0, 0])
          # plt.suptitle('TEST FILE: '+file_name+'\nSLICE NUMBER: '+str(i)+'\n'+
          #             'MODEL USED: '+model_path.split(main_path)[-1]+'\n'+
          #             'DEVICE: '+my_device.upper()+'\n'+
          #             'THRESHOLD: '+str(threshold_of_error)+' [0 --> 1]\n'+
          #             'ACCURACY: '+str(accuracy)+'%\n'+
          #             'TIMESTAMP: '+timenow(),ha='left',va='top',x=0.11,y=.92)


          if accuracy > 88 and accuracy < 100:
            # clear()
            file_name = model_path.split(main_path)[-1].split('/')[2]
            print(file_name)
            plt.savefig(main_path+'TESTED_DATA/'+file_name+sep+timenow()+'.png',bbox_inches='tight',dpi=600)
            plt.show()
            print('SAVED VALIDATION....')

##############JUST PLOTTING

### JUST FOR VISUAL REPRESENTATION.  WE ARE SHOWING HOW THE INDIVIDUAL IMAGES ARE PREPROCESSED IN ORDER TO BEST SUIT THE MODEL.

def shows_processed_image_pipeline_for_a_random_image():
  import numpy as np, nibabel
  from matplotlib import pyplot as plt
  from monai.transforms import(
      Compose,
      AddChanneld,
      LoadImaged,
      Resized,
      ToTensord,
      Spacingd,
      Orientationd,
      ScaleIntensityRanged,
      CropForegroundd,
  )
  from monai.data import DataLoader, Dataset, CacheDataset
  from monai.utils import set_determinism
  from monai.utils import first
  in_dir = traindir
  main_path = '/content/drive/MyDrive/thesis_dataset/'
  livpath = main_path+'Liver/Task03_Liver/'
  mypath = main_path+'test_path/'
  trpath = main_path+'imagesTr_path'
  labels = main_path+'labels_path'
  tspath = main_path+'imagesTs_path'
  folder = 'imagesTr'
  sixtyfournifties = main_path+'groups_of_64_nifti/'+folder+'/'



  inc = 0
  index = np.random.randint(0,366)

  slice_number1 = np.random.randint(0,64)
  # print(index,file_index,slice_number1)


  path_test_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))[index:]
  path_test_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))[index:]
  test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]

  file_index = np.random.randint(0,len(test_files)-1)
  test_files = test_files[file_index-1:file_index]
  file_name1 = test_files[0]['vol']


  # file_name1

  slice_number = int(file_name1.split('.nii.gz')[0].split('_')[-1])*64+slice_number1


  # files = os.listdir(sixtyfournifties)
  test_image1 = nibabel.load(file_name1).get_fdata()
  # test_image2 = nibabel.load(file_name2).get_fdata()
  test_image1 = correct_orient(test_image1)
  # test_files = [{'vol':file_name1,'seg':file_name1.replace('Volumes','Segmentation')}]

  plt = ''
  ax = ''
  fig = ''
  from matplotlib import pyplot as plt

  fig,ax = plt.subplots(figsize=(23,14))

  plt.rc('font',family='serif',size=35)
  COLOR = [0,0,0]
  # ax.tick_params(size=14)
  # ax.yticks(fontsize=14)
  # plt.axis('on')
  plt.rcParams['axes.labelcolor'] = COLOR
  plt.rcParams['xtick.labelsize'] = 26
  plt.rcParams['ytick.labelsize'] = 0
  plt.rcParams['xtick.color'] = COLOR
  plt.rcParams['ytick.color'] = COLOR


  plt.subplot(2,4,1)

  str0 = '1. Loading\nSlice'
  plt.imshow(test_image1[:,:,slice_number1],cmap='gray')
  plt.xticks([0, test_image1.shape[0]])
  plt.yticks([0, test_image1.shape[0]-1])
  plt.title(str0)


  str1 = '2. Adding an\nextra channel'
  plt_number = 2
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            # Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            # Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            # ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True),
            # CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            # Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)

  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])


  str1 = '3. Bilinear\nSpacing'
  plt_number = 3
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            # Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            # ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True),
            # CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            # Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])

  str1 = '4. Correcting\nOrientation'
  plt_number = 4
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            # ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True),
            # CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            # Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])

  str1 = '5. Scaling\nIntensity'
  plt_number = 5
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True),
            # CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            # Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])

  str1 = '6. Croping\nForeground'
  plt_number = 6
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            # Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  a,b = test_image2.shape

  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  # plt.xticks([0, test_image2.shape[1]])
  # plt.yticks([test_image2.shape[0]-1-test_image2.shape[1], test_image2.shape[0]-1])
  plt.xticks([0,a])
  plt.yticks([test_image2.shape[0]-1-b,test_image2.shape[0]-1])
  str1 = '7. Resizing\n'
  plt_number = 7
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])

  str1 = '8. Processed\nSlice'
  plt_number = 8
  test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            Resized(keys=["vol", "seg"], spatial_size=[128,128,64]),
            ToTensord(keys=["vol", "seg"]),
        ]
    )
  test_ds = Dataset(data=test_files, transform=test_transforms)
  test_loader = DataLoader(test_ds, batch_size=1)
  test_patient = first(test_loader)
  test_image2 = test_patient['vol'][0,0,:,:,slice_number1].numpy()
  test_image2 = correct_orient(test_image2)
  plt.subplot(2,4,plt_number)
  plt.imshow(test_image2,cmap='gray')
  plt.title(str1)
  plt.xticks([0, test_image2.shape[0]])
  plt.yticks([0, test_image2.shape[0]-1])

  # plt.suptitle('Processing '+file_name1.split(trvol)[-1]+' at slice number '+str(slice_number1))
  plt.savefig(main_path+'TESTED_DATA/PREPROCESSING IMAGE '+timenow()+'.png',bbox_inches='tight',dpi=600)
  plt.show()
  print('SAVED PREP....')
  plt.show()

  # return accuracy

CTRL+START
#ORGAN SEGMENTATION (LIVER) OF 3D MEDICAL IMAGES USING PYTORCH & MONAI
###GROUP 4, Md. Salim Shahed Shajid [2017338016], MD. Ashraf Hossain Ifty [2017338042]

thesis_database folder: https://drive.google.com/drive/folders/1VBjfxsrTDK6jChNYg4QCVIurLydUP0fB

Add this folder as a shortcut in the root folder 'My Drive.'

#FOLDERS OF INTEREST
####TRAINER -- contains the data for training and testing.
####TESTED_DATA -- contains the validation of the models.

#MODEL folders
1. MODEL
2. MODELscratch
3. MODELscratch_2


#Summary

Dataset from the Decathlon medical image website [http://medicaldecathlon.com/] was collected. It is the Task03_Liver.tar in the folder. Data were extracted [thesis_database/Liver] and then because of uneven slices for each CT scan file [thesis_database/imagesTr_path], we sliced the standalone files in packets of 64 sliced nifti files [.nii.gz]. The segmented slices [thesis_database/labels_path] were also packeted in a similar fashion.
After this, we discarded the nifti files that had no segmentation [the corresponding labels file had no data] and put them in the TRAINER folder. The actual slices were copied to the [thesis_database/TRAINER/TrainVolumes] folder, and the segmentation [labels] were copied to the [thesis_database/TRAINER/TrainSegmentation] folder.

Then we took files from the TrainVolumes folder, and processed them further to make the training less time consuming. The processing part was to resize the image from [512,512] pixel to [128,128], the foreground [pixels with significant data] and background [pixels with not enough data] were saturated, the orientation was normalized in a consistent permutation i.e. RAS orientation. Then, the image's voxel width and height, were changed to 1.5 to make the processing faster and we further contrasted the images foreground to make the features more prominent.

We split the TrainVolume data into random slices for Training and Testing the models accuracy. After that the data were cached into the gpu memory for faster training time, and now, we have a few trained models at various epochs.

In the validation part of this system, the model takes in the processed image and predicts a mask where the liver should be. We compare this prediction with the actual label and determine how accurate the prediction was. The comparism is a simple XOR operation between the two images. So, when both pixel value is 1, the result is 0. This why we find the inaccuracy of the prediction and the target.



In [ ]:
###BEFORE DOING ANYTHING, BTRL+F8, TO RUN ABOVE CELLS AND THEN RUN THIS CELL
###AND MOUNT DRIVE
###TO SEE IF EVERYTHING IS IN ORDER
###TRAINER FOLDER CANNOT BE 0 BYTES
dont_check = ['Liver','imagesTr_path','imagesTs_path','kaggler',
              'labels_path','groups_of_64','groups_of_64_nifti','groups_of_64_nifti_non_empty','model_results','fusemachine kaggle']
for files in os.listdir(main_path):
  if files not in dont_check:
    try:
      print(files,sep,format_size(get_dir_size(main_path+files)))
    except NotADirectoryError:
      getfilesize(main_path+files,'')
    # print(files)


In [ ]:
#STARTING POINT,
#RUN THIS CELL TO LOAD ALL THE DATA IN THE MEMORY FOR FASTER TRAINING
#YOU WILL GET RUNTIMEERROR THE FIRST TIME, JUST RE-RUN THIS CELL
#AND AFTER THE DATA IS LOADED ONTO THE MEMORY RUN THE NEXT CELL TO TRAIN THE MODEL

from monai.networks.nets import UNet
import sys

from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss
import torch

#########################NOTEBOOKS IN PARALLEL OPERATION#############ONLY CHANGE THESE VARIABLES CLONE NUMBER FROM 1 TO 5, AND DESIRED EPOCHS
CLONE_NUMBER = 1
epochs = 100
########################################################


model_dir = kfold+'DATA_FOLD_'+str(CLONE_NUMBER)+'/'

print(model_dir)

size = 128
data_caches_for_different_kfold_splits = []

train_files = sorted(glob(os.path.join(data_dir,"TrainVolumes","*.nii.gz")))
# train_files = sorted(os.listdir(data_dir+'TrainVolumes'))
train_files_segmentation = sorted(glob(os.path.join(data_dir,"TrainSegmentation","*.nii.gz")))

kfold_database = to_json({},'kfold_database','read')['FOLD_'+str(CLONE_NUMBER)]

kfold_train = [int(x) for x in kfold_database['kfold_train'].split(', ')]

kfold_test = [int(x) for x in kfold_database['kfold_test'].split(', ')]

if os.path.exists(model_dir+'graphs/') is False:
  os.mkdir(model_dir+'graphs/')

try:
  epoch_done = len(np.load(model_dir+'loss_train.npy'))
except Exception as e:
  epoch_done = 0


try:
  print(data_in)

except NameError:


  if epochs-epoch_done >0:
    data_in = prepare(data_dir, kfold_train, kfold_test,cache=True,spatial_size=[size,size,64])  #### THIS ONE TAKES DATA IN THE CACHE, TO MAKE THE TRAINING FAST
    clone_k = CLONE_NUMBER

if clone_k != CLONE_NUMBER and epochs-epoch_done>0:
  data_in = prepare(data_dir, kfold_train, kfold_test,cache=True,spatial_size=[size,size,64])  #### THIS ONE TAKES DATA IN THE CACHE, TO MAKE THE TRAINING FAST
  clone_k = CLONE_NUMBER

print(model_dir)

In [ ]:
#ACTUAL TRAINER, DOESNT MATTER IF TRAINING GETS INTERRUPTED, IT WILL DO THE TRAIN FROM THE
#NEXT EPOCH, AS LONG AS NOTHING IS TOUCHED HERE
device = torch.device(my_device)
learn_rate =  1e-5

model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)
go = ''
if os.path.exists(model_dir+'best_metric_model.pth'):
  print('RESUMING FROM MODEL'+sep+model_dir.split('/')[-2])
  model.load_state_dict(torch.load(
        os.path.join(model_dir, "best_metric_model.pth"),map_location=torch.device(my_device)))#### to resume for a specific model

  print(epoch_done,'epochs done.......')
  if epochs-epoch_done > 0:
    print('Remaining epochs',epochs-epoch_done)

    # model.eval()
    go='1'
else:
  go = '1'
  epoch_done = 0

# loss_function = DiceCELoss(to_onehot_y=True, sigmoid=True, squared_pred=True, ce_weight=calculate_weights(3.32088658e+08, 2.54757580e+07).to(device))
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)

if go not in empty_list:
  print('STARTING TRAINING IN 5 SECONDS....')
  time.sleep(5)
  train(model, data_in, loss_function, optimizer, epochs-epoch_done, model_dir)
# except ZeroDivisionError:
#   clear()
#   train(model, data_in, loss_function, optimizer, 600, model_dir)

In [ ]:
####DELETE (FAULTY) EPOCH INDEX
CL_NUMBER = 5
fault_in_the_last = 0
delete = 0 ################################### MAKE SURE THIS IS ALWAYS 0


model_dir = kfold+'/DATA_FOLD_'+str(CL_NUMBER)+'/'

if delete == 85695436:
  delete_confirm = 1
  files = ['loss_train','metric_train','loss_test','metric_test']
else:
  files = ['loss_train']
  delete_confirm = ''


for f in files:
  if fault_in_the_last == 0:

    a = np.load(model_dir+f+'.npy')
  else:
    a = np.load(model_dir+f+'.npy')[0:-fault_in_the_last]


  if delete_confirm == 1:
    print('reforming',model_dir+f+'.npy')
    np.save(model_dir+f+'.npy',arr=a)


  print(a,'\n')

  print(len(a))
  print('-------------------')



#only the functions in this notebook are necessary, the other codes were just to visualize the data. or to segment the files to various folders for ease of use. to see those go to:

##https://colab.research.google.com/drive/1VCMNqVPn8XP3Onmvf2ZAjEh1vogZqfK6